In [4]:
# | default_exp backend.llms.client

In [1]:
# | export
import os
from typing import Dict, Any, List, Optional, Iterator
from openai import OpenAI
from rich.console import Console
from rich.markdown import Markdown

In [7]:
# | export
from agentic.configs.loader import get_model_config
from agentic.backend.llms.response_processor import ResponseProcessor
from agentic.backend.llms.streaming_handler import StreamingHandler

ModuleNotFoundError: No module named 'agentic.backend.llms'

In [ ]:
# | export
class LLMClient:
    """Enhanced LLM client"""
    
    def __init__(self, model: Optional[str] = None, base_url: Optional[str] = None, 
                 api_key: Optional[str] = None):
        # Load config defaults
        model_config = get_model_config()
        settings_config = get_settings_config()
        
        # Set parameters with fallbacks
        self.model = model or model_config.get('name', 'qwen3:8b')
        self.base_url = base_url or model_config.get('url', 'http://localhost:11434/v1')
        self.api_key = api_key or model_config.get('api_key', 'ollama')

        
        # Initialize OpenAI client
        self.client = OpenAI(base_url=self.base_url, 
                             api_key=self.api_key,
                             timeout=timeout)
        
        # Initialize processors
        self.response_processor = ResponseProcessor()
        self.streaming_handler = StreamingHandler()
        
        # Validate connection
        self._validate_connection()
    
    def _validate_connection(self):
        """Validate LLM connection"""
        try:
            # Test with a simple request
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": "test"}],
                max_tokens=1,
                timeout=5
            )
            return True
        except Exception as e:
            print(f"⚠️ LLM connection warning: {e}")
            return False
    
    def create_completion(self, messages: List[Dict[str, Any]], 
                         tools: Optional[List[Dict]] = None,
                         stream: bool = True, **kwargs) -> Any:
        """Create chat completion with optional tools"""
        completion_params = {
            "model": self.model,
            "messages": messages,
            "stream": stream,
            **kwargs
        }
        
        if tools:
            completion_params["tools"] = tools
            completion_params["tool_choice"] = "auto"
        
        try:
            return self.client.chat.completions.create(**completion_params)
        except Exception as e:
            raise RuntimeError(f"LLM completion failed: {e}")
    
    def process_response(self, response: Any, console: Optional[Console] = None) -> Dict[str, Any]:
        """Process non-streaming response"""
        return self.response_processor.process_response(response, console)
    
    def handle_streaming_response(self, response: Iterator, console: Optional[Console] = None) -> Dict[str, Any]:
        """Handle streaming response"""
        return self.streaming_handler.handle_streaming_response(response, console)
    
    def get_model_info(self) -> Dict[str, Any]:
        """Get information about the current model"""
        return {
            "model": self.model,
            "base_url": self.base_url,
            "api_key_set": bool(self.api_key),
            "connection_valid": self._validate_connection()
        }
